In [1]:
from openhunt.logparser import winlogbeat
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [2]:
win = winlogbeat()
spark = SparkSession.builder.appName("Process").config("spark.sql.caseSensitive", "True").getOrCreate()

In [3]:
process_hollowing = win.extract_nested_fields("process_hollowing_2019-10-17111939.json",spark)

[+] Processing a Spark DataFrame..
[+] Reading Mordor file..
[+] Processing Data from Winlogbeat version 7..
[+] DataFrame Returned !


In [4]:
process_hollowing.createOrReplaceTempView("process_hollowing")

In [14]:
ProcessHollowing = spark.sql(
'''
SELECT 
    c.computer_name, 
    c.User,
    c.ParentImage,
    d.TargetImage,
    c.LogonId,
    ltrim('0x', d.GrantedAccess) as granted_access
FROM process_hollowing b  
JOIN process_hollowing c
    ON c.Image = b.NewProcessName
    AND c.channel = "Microsoft-Windows-Sysmon/Operational"
    AND c.event_id = 1
    AND b.SubjectUserName = (substring_index(c.User, '\\\\', -1))
JOIN process_hollowing d
    ON c.ProcessGuid = d.TargetProcessGUID
    AND d.channel = "Microsoft-Windows-Sysmon/Operational"
    AND d.event_id = 10
    AND c.ParentProcessGuid = d.SourceProcessGUID
    AND c.Image = d.TargetImage
JOIN process_hollowing e
    ON e.ProcessGuid = c.ParentProcessGuid
    AND e.channel = "Microsoft-Windows-Sysmon/Operational"
    AND e.event_id = 5
    AND e.Image = d.SourceImage
    AND c.ParentImage = e.Image
JOIN process_hollowing f
    ON f.ProcessGuid = c.ParentProcessGuid
    AND f.channel = "Microsoft-Windows-Sysmon/Operational"
    AND f.event_id = 7
    AND f.Image = c.ParentImage
    AND f.Image = f.ImageLoaded
WHERE b.channel = "Security"
    AND b.event_id = 4688
'''
)


In [15]:
df = ProcessHollowing.select("computer_name", conv(ProcessHollowing.granted_access, 16, 10).alias("granted_access"), "User", "ParentImage", "TargetImage", "LogonId")

In [16]:
df.createOrReplaceTempView("df")

In [17]:
process_hollowing_minimal_privileges = spark.sql(
'''
SELECT * FROM df WHERE ((INT(granted_access) & 2312) == 2312) -- 40 is decimal for 0x0028. The minimal privileges you need to access process handle
'''
).show(1,False)

+------------------+--------------+-----------+----------------------------------------+-------------------------------+--------+
|computer_name     |granted_access|User       |ParentImage                             |TargetImage                    |LogonId |
+------------------+--------------+-----------+----------------------------------------+-------------------------------+--------+
|win10.marvel.local|2097151       |MARVEL\thor|C:\ProcessHollowing\ProcessHollowing.exe|C:\Windows\SysWOW64\svchost.exe|0x8e86da|
+------------------+--------------+-----------+----------------------------------------+-------------------------------+--------+
only showing top 1 row

